In [43]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import csv
#import pandasql as ps
import matplotlib.pyplot as plt
#import shapefile as shp
import seaborn as sns
import random
import time

import statistics

#temporal pattern clustering
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from tslearn.utils import to_time_series_dataset
from sklearn import metrics
from scipy.cluster.hierarchy import fcluster
from scipy.stats import chisquare
from scipy.stats import wasserstein_distance
from sklearn.metrics import mean_squared_error


import time
import scipy.sparse
from scipy.stats import entropy
from sys import exit

import itertools
from itertools import permutations

import xml.etree.ElementTree as ET

import geopandas as gpd

In [2]:
path = "G:/My Drive/2021/Bias/sumo_simulation/appsim"
os.chdir(path)

In [3]:
WH_path = []
for i in os.listdir():
    if 'WH' in i:
        WH_path.append(i)
print(WH_path)

['WH_max_ts.csv', 'WH_index_ts.csv', 'WH_max_t.csv', 'WH_index_t.csv', 'WH_max_ts_1000.csv', 'WH_index_ts_1000.csv']


In [4]:
fcd_out = pd.read_csv('fcd_out_cur.csv', index_col=0)
print('fcd_out df', fcd_out.head(3))
selected_taz = np.unique(fcd_out['taz'])
fcd_out.columns = ['id', 'taz_id', 'time', 'tl_id', 'time_5', 
                   'sim_obs', 'sim_obs_ts', 'obs','user']

fcd_out df               id  taz  time  tl_id  time_5  sim_obs  sim_obs_ts  obs  newid
0    10_41979657  506     0  506_0       0        0           0    1   1237
1    13_49194874  440     0  440_0       0        1           0    1   5170
2  21_47580428_0  630     0  630_0       0        0           0    1  13841


In [5]:
#convert WH to csv
def get_taz_dict(tazs):
    taz_dict = {}
    for i in range(len(tazs)):
        taz_dict[i] = tazs[i]
    return(taz_dict)

def add_taz_col(tazs, dt, convert_col):
    taz_dict = get_taz_dict(tazs)
    convert_taz = lambda x: taz_dict[x]
    dt['taz_id'] = dt[convert_col].apply(convert_taz)
    return(dt)

def get_WH_info(index_path, maxval_path, selected_taz):
    WH_index = pd.read_csv(index_path,  index_col=0).to_numpy()
    WH_maxval = pd.read_csv(maxval_path,  index_col=0).to_numpy()
    index_mx = 0
    
    for i in range(WH_index.shape[0]):
        indexval = WH_index[i]
        maxval = WH_maxval[i]
        time_t = np.nonzero(indexval)[0]
        #print(time_t)
        #print(len(time_t))
        if len(time_t) > 1:
            taz_index = indexval[time_t]
            obs_val = maxval[time_t]
            userid = np.ones(len(time_t))*i
            if index_mx == 0:
                data_info = np.array([userid, time_t, taz_index, obs_val])
                data_info = data_info.T
                #print(data_info)
                index_mx += 1
            else:
                data_info_cur = np.array([userid, time_t, taz_index, obs_val])
                data_info_cur = data_info_cur.T
                #print(data_info_cur)
                data_info = np.vstack((data_info, data_info_cur))
                
    data_info = pd.DataFrame(data_info)
    data_info.columns = ['user', 'time_5', 'fake_taz', 'obs']
    data_info = add_taz_col(selected_taz, data_info, 'fake_taz')
    
    return(data_info)

In [153]:
#previous data processing
#dt_WH_ts = get_WH_info('WH_index_ts.csv', 'WH_max_ts.csv', selected_taz)
#dt_WH_t = get_WH_info('WH_index_t.csv', 'WH_max_t.csv', selected_taz)

#processing the non_zeros
dt_WH_ts = dt_WH_ts[dt_WH_ts.obs>0.001]
print('WH based on ts shape', dt_WH_ts.shape)

dt_WH_t = dt_WH_t[dt_WH_t.obs>0.001]
print('WH based on t shape', dt_WH_t.shape)

print('dt_WH_t', dt_WH_t.obs.describe())
print('dt_WH_ts', dt_WH_ts.obs.describe())

WH based on ts shape (186420, 5)
WH based on t shape (214044, 6)
dt_WH_t count    214044.000000
mean          0.034199
std           0.086039
min           0.001000
25%           0.002804
50%           0.007956
75%           0.027314
max           1.604113
Name: obs, dtype: float64
dt_WH_ts count    186420.000000
mean          0.029510
std           0.086424
min           0.001000
25%           0.002192
50%           0.005748
75%           0.019183
max           1.696141
Name: obs, dtype: float64


In [157]:
#previous ids
userid_wh_t = np.unique(dt_WH_t.user)
userid_wh_ts = np.unique(dt_WH_ts.user)

print(len(userid_wh_t))
print(len(userid_wh_ts))

12681
16089


In [6]:
def check_overlapped_num(pred_dt, act_dt):
    overlapped_index = []
    for index_ in pred_dt['check_index'].values:
        if index_ in act_dt['check_index'].values:
            overlapped_index.append(index_)
    return(overlapped_index)
    
def get_overlapped_index(index_path, maxval_path, selected_taz, act_dt):
    pred_dt = get_WH_info(index_path, maxval_path, selected_taz)
    print('pred_dt shape info', pred_dt.shape)
    
    act_dt['check_index'] = act_dt['user'].astype(str)+'_'+act_dt['taz_id'].astype(str)+'_'+act_dt['time_5'].astype(str)
    pred_dt['user'] = pred_dt['user'].astype(int)
    pred_dt['time_5'] = pred_dt['time_5'].astype(int)
    
    pred_dt['check_index'] = pred_dt['user'].astype(str)+'_'+pred_dt['taz_id'].astype(str)+'_'+pred_dt['time_5'].astype(str)
    
    #check overlapped
    overlapped_index = check_overlapped_num(pred_dt, act_dt)
    print('num overlapped index', len(overlapped_index))
    
    pred_dt = pred_dt[pred_dt['check_index'].isin(overlapped_index)]
    return(pred_dt)

In [7]:
#previous test pred_WH = get_overlapped_index(dt_WH_t, fcd_out)
dt_WH_t = get_overlapped_index('WH_index_t.csv', 'WH_max_t.csv', selected_taz, fcd_out)
dt_WH_ts = get_overlapped_index('WH_index_ts.csv', 'WH_max_ts.csv', selected_taz, fcd_out)
dt_WH_ts_1000 = get_overlapped_index('WH_index_ts_1000.csv', 'WH_max_ts_1000.csv', selected_taz, fcd_out)

pred_dt shape info (5632776, 5)
num overlapped index 12929
pred_dt shape info (5873446, 5)
num overlapped index 11486
pred_dt shape info (5523140, 5)
num overlapped index 31013


In [28]:
fcd_out['check_index'].shape

(138268,)

In [21]:
dt_WH_t['check_index'].values[0]

'3_630_14'

In [55]:
dt_WH_ts.columns

Index(['user', 'time_5', 'fake_taz', 'obs', 'taz_id', 'check_index', 'obs_ts'], dtype='object')

In [40]:
def merge_df(pred_dt, sim_dt, pred_obs_col):
    pred_dt[pred_obs_col] = 1
    pred_ = pred_dt[[pred_obs_col,'check_index']]
    df = pd.merge(pred_, sim_dt, on='check_index', how='right')
    df[pred_obs_col] = df[pred_obs_col].replace(np.nan, 0)
    return(df)

In [56]:
def merge_df_predval(pred_dt, sim_dt, pred_obs_col):
    pred_dt[pred_obs_col] = pred_dt['obs']
    pred_ = pred_dt[[pred_obs_col,'check_index']]
    df = pd.merge(pred_, sim_dt, on='check_index', how='right')
    df[pred_obs_col] = df[pred_obs_col].replace(np.nan, 0)
    return(df)

In [41]:
fcd_out = merge_df(dt_WH_t, fcd_out, 'obs_t')
fcd_out = merge_df(dt_WH_ts, fcd_out, 'obs_ts')
fcd_out = merge_df(dt_WH_ts_1000, fcd_out, 'obs_ts_1000')

In [57]:
fcd_out = merge_df_predval(dt_WH_t, fcd_out, 'obsp_t')
fcd_out = merge_df_predval(dt_WH_ts, fcd_out, 'obsp_ts')
fcd_out = merge_df_predval(dt_WH_ts_1000, fcd_out, 'obsp_ts_1000')

In [42]:
fcd_out.columns

Index(['obs_ts_1000', 'check_index', 'obs_ts', 'obs_t', 'id', 'taz_id', 'time',
       'tl_id', 'time_5', 'sim_obs', 'sim_obs_ts', 'obs', 'user'],
      dtype='object')

In [47]:
def get_rmse(col1, col2, df):
    val1 = df[col1].values
    val2 = df[col2].values
    print(col1+' vs.'+col2, mean_squared_error(val1, val2))

In [58]:
print('pred comparison with original data')
get_rmse('obs_ts_1000', 'obs', fcd_out)
get_rmse('obs_ts', 'obs', fcd_out)
get_rmse('obs_t', 'obs', fcd_out)

get_rmse('obsp_ts_1000', 'obs', fcd_out)
get_rmse('obsp_ts', 'obs', fcd_out)
get_rmse('obsp_t', 'obs', fcd_out)

print('\n')
print('sim app-based data with original data')
get_rmse('sim_obs', 'obs', fcd_out)
get_rmse('sim_obs_ts', 'obs', fcd_out)

print('\n')
print('sim app-based data with pred comparison')
get_rmse('obs_ts_1000', 'sim_obs_ts', fcd_out)
get_rmse('obs_ts', 'sim_obs_ts', fcd_out)
get_rmse('obs_t', 'sim_obs', fcd_out)

get_rmse('obsp_ts_1000', 'sim_obs_ts', fcd_out)
get_rmse('obsp_ts', 'sim_obs_ts', fcd_out)
get_rmse('obsp_t', 'sim_obs', fcd_out)

pred comparison with original data
obs_ts_1000 vs.obs 0.7757037058466167
obs_ts vs.obs 0.9169294413747215
obs_t vs.obs 0.9064931871438077
obsp_ts_1000 vs.obs 0.9041647637372819
obsp_ts vs.obs 0.9843735433056967
obsp_t vs.obs 0.9064931871438077


sim app-based data with original data
sim_obs vs.obs 0.5702042410391414
sim_obs_ts vs.obs 0.6399745421934214


sim app-based data with pred comparison
obs_ts_1000 vs.sim_obs_ts 0.34620447247374664
obs_ts vs.sim_obs_ts 0.35228686319321895
obs_t vs.sim_obs 0.40210316197529433
obsp_ts_1000 vs.sim_obs_ts 0.319461189684709
obsp_ts vs.sim_obs_ts 0.3503101979423612
obsp_t vs.sim_obs 0.40210316197529433


In [ ]:
pred comparison with original data
obs_ts_1000 vs.obs 0.7757037058466167
obs_ts vs.obs 0.9169294413747215
obs_t vs.obs 0.9064931871438077
obsp_ts_1000 vs.obs 0.9041647637372819
obsp_ts vs.obs 0.9843735433056967
obsp_t vs.obs 0.9064931871438077


sim app-based data with original data
sim_obs vs.obs 0.5702042410391414
sim_obs_ts vs.obs 0.6399745421934214


sim app-based data with pred comparison
obs_ts_1000 vs.sim_obs_ts 0.34620447247374664
obs_ts vs.sim_obs_ts 0.35228686319321895
obs_t vs.sim_obs 0.40210316197529433
obsp_ts_1000 vs.sim_obs_ts 0.319461189684709
obsp_ts vs.sim_obs_ts 0.3503101979423612
obsp_t vs.sim_obs 0.40210316197529433